# Deep Learning segmentation suite dessigned for 2D microscopy image segmentation
---

<font size = 4>The corresponging **[GitHub repository](https://github.com/esgomezm/microscopy-dl-suite-tf)**, developed by the authors of the paper. 

---
<font size = 4>**Please cite the corresponding paper** 

## Install required libraries
<font size = 4>In case it returns "restart session" at the end fo the installation, please click it and start from the beginning.

In [1]:
!git clone https://github.com/esgomezm/microscopy-dl-suite-tf
!pip3 install -r microscopy-dl-suite-tf/dl-suite/requirements.txt

Cloning into 'microscopy-dl-suite-tf'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 220 (delta 102), reused 60 (delta 29), pack-reused 0
Receiving objects: 100% (220/220), 4.27 MiB | 4.01 MiB/s, done.
Resolving deltas: 100% (102/102), done.
     |████████████████████████████████| 47.6 MB 74 kB/s 
     |████████████████████████████████| 516.2 MB 4.4 kB/s 
     |████████████████████████████████| 48.4 MB 1.5 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 230 kB 47.9 MB/s 
     |████████████████████████████████| 454 kB 55.7 MB/s 
     |████████████████████████████████| 3.0 MB 11.5 MB/s 
     |████████████████████████████████| 2.9 MB 50.9 MB/s 
     |████████████████████████████████| 40 kB 6.0 MB/s 
     |████████████████████████████████| 1.0 MB 53.7 MB/s 
     |████████████████████████████████| 802 kB 55.9 MB/s 
     |███████

## Alternative: Mount your drive and work with the information there

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
path_code = "Path where the code is stored"
os.chdir(path_code)
!pip3 install -r microscopy-dl-suite-tf/dl-suite/requirements.txt

# Download example data

It will download the example data from a [Zenodo repository](https://zenodo.org/record/5777994) and unzip it.

If you use this data, please cite the corresponding publication

In [5]:
!wget https://zenodo.org/record/5777994/files/HT1080WT_PHASECONTRAST.zip

--2021-12-14 20:14:24--  https://zenodo.org/record/5777994/files/HT1080WT_PHASECONTRAST.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3557453821 (3.3G) [application/octet-stream]
Saving to: ‘HT1080WT_PHASECONTRAST.zip’

ONTRAST.zip           2%[                    ]  95.33M  12.0MB/s    eta 4m 16s ^C


In [6]:
!unzip HT1080WT_PHASECONTRAST.zip?download=1

Archive:  HT1080WT_PHASECONTRAST.zip?download=1
   creating: HT1080WT_PHASECONTRAST/
   creating: HT1080WT_PHASECONTRAST/val/
  inflating: HT1080WT_PHASECONTRAST/val/original_videos_ordered.csv  
   creating: HT1080WT_PHASECONTRAST/val/inputs/
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_018.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_022.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_025.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_021.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_026.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_023.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_024.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_027.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_020.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_019.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_017.tif  
  inflating: HT1080WT_PHASECONTRAST/val/inputs/raw_016.tif  
  inflating: HT1080WT_PH

# Train

## Set up a new training from scratch

In [13]:
import json
import time
import os

json_template ='/content/microscopy-dl-suite-tf/examples/config/config_template.json'

training_data = "/content/HT1080WT_PHASECONTRAST/train"
validation_data = "/content/HT1080WT_PHASECONTRAST/val"
test_data = "/content/HT1080WT_PHASECONTRAST/test"
# The test data will be store following the cell tracking challenge order, so we need to get some order that corresponds to the CTC GT.
test_data_video_relation = "/content/HT1080WT_PHASECONTRAST/test/stack2im/videos2im_relation.csv"
## OPTIONS FOR MODEL ARCHITECTURES. TRAINING DATA NEEDS TO CORRESPOND TO THE 
# INPUT-OUTPUT FORMAT (2D IMAGES OR 2D VIDEO FILES)
# ---------------------------------------------------------

# - 'mobilenet_mobileunet_lstm': A pretrained mobilenet in the encoder with skip connections to the decoder of a mobileunet and a ConvLSTM layer at the end that will make the entire architecture recursive.
# - 'mobilenet_mobileunet': A pretrained mobilenet in the encoder with skip connections to the decoder of a mobileunet (2D).
# - 'unet_lstm': 2D U-Net with ConvLSTM units in the contracting path.
# - 'categorical_unet_transpose': 2D U-Net for different labels ({0}, {1}, ...) with transpose convolutions instead of upsampling.
# - 'categorical_unet_fc_dil': 2D U-Net for different labels ({0}, {1}, ...) with fully connected dilated convolutions.
# - 'categorical_unet_fc': 2D U-Net for different labels ({0}, {1}, ...) with fully connected convolutions.
# - 'categorical_unet': 2D U-Net for different labels ({0}, {1}, ...).
# - 'unet' or "None": 2D U-Net with a single output.
model_architecture = "mobilenet_mobileunet_lstm"

## TRAINING PARAMETERS
training_epochs = 10
learning_rate = 0.00001
# 1 if we want to freeze the mobilenet encoder to 
# apply some transfer learning, 0 if we want to train the entire network:
freeze_pretrained_encoder = 0 
## TOTAL BATCH SIZE: patch_batch*bach_size
patch_batch = 1 # Number of patches cropped from each image on each epoch.
bach_size = 2 # Number of images to take on each epoch
spatial_input_shape = [256, 256]


# CREATE THE CONFIGURATION FILE
with open(json_template, "r") as jsonFile:
    data = json.load(jsonFile)
data["cnn_name"] = model_architecture
data["TRAINPATH"] = training_data
data["TESTPATH"] = test_data
data["VALPATH"] = validation_data
data["PATH2VIDEOS"] = test_data_video_relation
data["model_train_decoder_only"] = freeze_pretrained_encoder
data["model_lr"] = learning_rate
data["train_max_epochs"] = training_epochs
data["datagen_patch_batch"] = patch_batch
data["datagen_batch_size"] = bach_size
data["datagen_dim_size"] = spatial_input_shape

# CHOOSE A SEED FOR THE MODEL INITIALIZATION AND THE DATA GENERATOR TO REPRODUCE
# THE RESULTS
data["model_seed_initializer"] = 11
data["train_seed_initializer"] = 2

# CHOOSE THE PATH TO STORE THE MODEL
main_path = "/content/"
output_path = os.path.join(main_path, "my_fine_tuned_model")
if not os.path.exists(output_path):
  os.mkdir(output_path)

data["OUTPUTPATH"] = output_path

new_json = os.path.join(output_path, "my_config.json")

with open('/content/my_config.json', "w") as jsonFile:
    json.dump(data, jsonFile)

with open(new_json, "w") as jsonFile:
    json.dump(data, jsonFile)   
!python /content/microscopy-dl-suite-tf/dl-suite/train.py '/content/my_config.json'


Seed 11 fixed for model initialization
{'n_filters': 16, 'pools': 3, 'kernel_size': [3, 3], 'dilation_rate': 2, 'mobile_alpha': 0.35, 'time_windows': 5, 'lr': 1e-05, 'dropout': 0.01, 'activation': 'elu', 'last_activation': 'tanh', 'padding': 'same', 'kernel_initializer': 'glorot_uniform', 'lossfunction': 'sparse_cce', 'loss_tips': 'L1L2', 'metrics': 'accuracy', 'train_decoder_only': 0, 'category_weights': [1, 10], 'seed_initializer': 11}
2021-12-14 20:40:41.814196: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-14 20:40:41.831085: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 20:40:41.831790: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreCl

## Keep training a model from a certain checkpoint

We will use the same `config.json` as before (in the folder `my_fine_tuned_model`) and create a new folder to store the new results called `my_new_fine_tuned_model`.

In [14]:
# READ THE PREVIOUS JSON
json_file_path = "/content/my_fine_tuned_model/my_config.json"

# READ THE PREVIOUS CONFIGURATION FILE
with open(json_file_path, "r") as jsonFile:
    data = json.load(jsonFile)

output_path = "/content/my_new_fine_tuned_model"
data["OUTPUTPATH"] = output_path

# SETUP THE PATH FOR THE PRETRAINED MODEL
data["train_pretrained_weights"] = "/content/my_fine_tuned_model/checkpoints/mobilenet_mobileunet_lstmlast.hdf5"

# CHANGE ANY ADDITIONAL SET UPS FOR THE TRAINING
data["train_max_epochs"] = 5

if not os.path.exists(output_path):
  os.mkdir(output_path)

# STORE THE CONFIGURATION FILE IN THE NEW MODEL FOLDER
new_json = os.path.join(output_path, "my_config.json")
with open(new_json, "w") as jsonFile:
    json.dump(data, jsonFile)   

# STORE THE CONFIGURATION FILE IN CURRENT FOLDER TO CALL IT
with open('/content/my_config.json', "w") as jsonFile:
    json.dump(data, jsonFile)

!python /content/microscopy-dl-suite-tf/dl-suite/train.py '/content/my_config.json'


Seed 11 fixed for model initialization
{'n_filters': 16, 'pools': 3, 'kernel_size': [3, 3], 'dilation_rate': 2, 'mobile_alpha': 0.35, 'time_windows': 5, 'lr': 1e-05, 'dropout': 0.01, 'activation': 'elu', 'last_activation': 'tanh', 'padding': 'same', 'kernel_initializer': 'glorot_uniform', 'lossfunction': 'sparse_cce', 'loss_tips': 'L1L2', 'metrics': 'accuracy', 'train_decoder_only': 0, 'category_weights': [1, 10], 'seed_initializer': 11}
2021-12-14 21:06:51.205437: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-14 21:06:51.227707: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-14 21:06:51.228420: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreCl

## Run the training using a .json file that contains all the details

In [ ]:
!python /content/microscopy-dl-suite-tf/dl-suite/train.py '/content/microscopy-dl-suite-tf/examples/config/config_template.json' 